In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

open_ai_key = os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

Componentes de RAG

In [4]:
# Modelo de embeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

model = ChatOpenAI(model="gpt-4o", temperature=0)
embeddings_model = OpenAIEmbeddings()

In [5]:
from langchain_community.document_loaders import PyPDFLoader

# cargar documentos
loader = PyPDFLoader("example_data/meeting_data.pdf")
paper = loader.load()

In [ ]:
%pip install -U faiss-cpu

In [6]:
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
chunks_meeting_data = text_splitter.split_documents(paper)
faiss_db_paper = FAISS.from_documents(chunks_meeting_data, embeddings_model)
info_meetings_retriever= faiss_db_paper.as_retriever()

Tools

In [59]:
# Crea una tool para recuperar información

from langchain.tools.retriever import create_retriever_tool

retrieval_tool = create_retriever_tool(info_meetings_retriever, "retrieval_tool", "Busca información sobre las reuniones")

In [8]:
retrieval_tool.invoke("¿Cuándo fue la reunion?")

'Reunión de planificación - 15 de agosto de 2024\nAsistentes:\n- Juan Pérez\n- María González\n- Luis Fernández\n- Ana Martínez\nAgenda:\n1. Revisión de los objetivos del trimestre pasado.\n2. Planificación de las próximas actividades.\n3. Asignación de responsabilidades.\n4. Preguntas y respuestas.\nConclusiones:\n- Se logró el 85% de los objetivos del trimestre pasado.\n- Se identificaron áreas de mejora en la comunicación interna.\n- Las actividades prioritarias para el próximo trimestre incluyen la mejora de procesos y la\ncapacitación del personal.\nLa próxima reunión está programada para el 20 de septiembre de 2024.'

In [ ]:
%pip install wikipedia

In [52]:
# Añade otras tools para tu agente

from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Tool para buscar información en Wikipedia
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [53]:
from langchain.tools import Tool

# Documenta tus tools 
wiki_tool = Tool.from_function(
    func=wikipedia.run,
    name="wiki_tool",
    description="Es un tool para buscar cierta info en Wikipedia")

# Lista de tools
tools = [wiki_tool, retrieval_tool] 

In [ ]:
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation 

# Toma una acción del agente y llama a la tool y devuelve el resultado
tool_executor = ToolExecutor(tools)

In [16]:

from langchain_core.messages import AIMessage
from langchain_core.tools import tool

from langgraph.prebuilt import ToolNode
tool_node = ToolNode(tools)

In [ ]:

invocation = ToolInvocation(tool="Documento para hacer Retrieval Augmented Generation", tool_input="Busca información sobre las reuniones")

tool_executor.invoke(invocation)

In [17]:

message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "Documento para hacer Retrieval Augmented Generation",
            "args": {"query": "¿Cuándo fue la reunion?"},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

tool_node.invoke({"messages": [message_with_single_tool_call]})

{'messages': [ToolMessage(content='Reunión de planificación - 15 de agosto de 2024\nAsistentes:\n- Juan Pérez\n- María González\n- Luis Fernández\n- Ana Martínez\nAgenda:\n1. Revisión de los objetivos del trimestre pasado.\n2. Planificación de las próximas actividades.\n3. Asignación de responsabilidades.\n4. Preguntas y respuestas.\nConclusiones:\n- Se logró el 85% de los objetivos del trimestre pasado.\n- Se identificaron áreas de mejora en la comunicación interna.\n- Las actividades prioritarias para el próximo trimestre incluyen la mejora de procesos y la\ncapacitación del personal.\nLa próxima reunión está programada para el 20 de septiembre de 2024.', name='Documento para hacer Retrieval Augmented Generation', tool_call_id='tool_call_id')]}

Agents

In [43]:
# Crear un Agente ReAct
from langchain import hub

prompt = hub.pull("hwchase17/react")

In [44]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


StateAgent

In [27]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator


class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

In [ ]:
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "Documento para hacer Retrieval Augmented Generation",
            "args": {"query": "¿Cuándo fue la reunion?"},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

tool_node.invoke({"messages": [message_with_single_tool_call]})

Graph - LangGraph

In [63]:
from typing import Annotated, Literal, TypedDict

from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

In [64]:

tools = [retrieval_tool, wiki_tool]
tool_node = ToolNode(tools)

In [65]:
model = ChatOpenAI(model="gpt-4o", temperature=0).bind_tools(tools)

In [66]:
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]

    if last_message.tool_calls:
        return "tools"
    return END

def call_model(state: MessagesState):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

In [67]:

workflow = StateGraph(MessagesState)

In [68]:
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

In [69]:
workflow.set_entry_point("agent")

In [70]:
workflow.add_conditional_edges(
    "agent",
    should_continue,
)

In [71]:
workflow.add_edge("tools", 'agent')

In [72]:
checkpointer = MemorySaver()

In [ ]:
app = workflow.compile(checkpointer=checkpointer)

In [ ]:
# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="¿Quiénes intervinieron en la reunión del 15 de agosto?")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

In [60]:










# Define the two nodes we will cycle between


# Set the entrypoint as `agent`
# This means that this node is the first one called


# We now add a conditional edge


# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.


# Initialize memory to persist state between graph runs


# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph




'En la reunión de planificación del 15 de agosto de 2024, intervinieron los siguientes asistentes:\n\n- Juan Pérez\n- María González\n- Luis Fernández\n- Ana Martínez\n\nLa agenda de la reunión incluyó la revisión de los objetivos del trimestre pasado, la planificación de las próximas actividades, la asignación de responsabilidades y una sesión de preguntas y respuestas.'

In [30]:
# Define los nodos
from langchain_core.agents import AgentActionMessageLog

def run_agent(data):
    inputs = data.copy()
    text = inputs['input']
    agent_outcome = agent_executor.invoke({"input":text})
    return {"agent_outcome": agent_outcome}

# Define la función que ejecuta las tools
def execute_tools(data):
    agent_output = data["agent_outcome"]
    if len(agent_output['intermediate_steps'])>=1 :
        agent_action = agent_output['intermediate_steps'][0][0]
        output = tool_node.invoke({"messages": [message_with_single_tool_call]})
        return {"intermediate_steps": [(agent_action, str(output))]}
    else:
        return {"intermediate_steps":[]}

# Lógica para decidir si el agente debe continuar o finalizar
def should_continue(data):

    if data["agent_outcome"]["output"] is not None:
        print(" **AgentFinish** " )
        return "end"
    else:
        print(" **continue...** " )
        return "continue"

In [33]:
from langgraph.graph import END, StateGraph

# Define un workflow de LangGraph
workflow = StateGraph(AgentState)

In [34]:
# Define los nodos
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

In [35]:
# Configura el punto de entrada
workflow.set_entry_point("agent")

In [36]:
# Añade los edges condicionales
workflow.add_conditional_edges(
    "agent",
    should_continue,
  
    {
        "continue": "action",
        "end": END,
    },
)

In [37]:
# Añade un normal edge
workflow.add_edge("action", "agent")

In [38]:
# Nos devuelve un runnable de LangChain
app = workflow.compile()

In [39]:
inputs = {"input": "¿Quiénes intervinieron en la reunión del 15 de agosto?"} 
outputs = app.invoke(inputs)



> Entering new AgentExecutor chain...
Para responder a esta pregunta, necesito buscar información específica sobre la reunión del 15 de agosto. Utilizaré la herramienta "Documento para hacer Retrieval Augmented Generation" para obtener detalles sobre esta reunión.

Action: Documento para hacer Retrieval Augmented Generation
Action Input: "reunión del 15 de agosto"Reunión de planificación - 15 de agosto de 2024
Asistentes:
- Juan Pérez
- María González
- Luis Fernández
- Ana Martínez
Agenda:
1. Revisión de los objetivos del trimestre pasado.
2. Planificación de las próximas actividades.
3. Asignación de responsabilidades.
4. Preguntas y respuestas.
Conclusiones:
- Se logró el 85% de los objetivos del trimestre pasado.
- Se identificaron áreas de mejora en la comunicación interna.
- Las actividades prioritarias para el próximo trimestre incluyen la mejora de procesos y la
capacitación del personal.
La próxima reunión está programada para el 20 de septiembre de 2024.Ahora tengo la infor

In [40]:
outputs['agent_outcome']['output']

'En la reunión del 15 de agosto de 2024 intervinieron Juan Pérez, María González, Luis Fernández y Ana Martínez.'